In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


# STEAM GAME RECOMMENDER
**(Content Based Recommendation)**

In [4]:
#DATA 
df = pd.read_csv("data/steam_games.csv")
df.head()

,url,types,name,desc_snippet,recent_reviews,all_reviews,release_date,developer,publisher,popular_tags,game_details,languages,achievements,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price,discount_price
0,https://store.steampowered.com/app/379720/DOOM/,app,DOOM,Now includes all three premium DLC packs (Unto...,"Very Positive,(554),- 89% of the 554 user revi...","Very Positive,(42,550),- 92% of the 42,550 use...","May 12, 2016",id Software,"Bethesda Softworks,Bethesda Softworks","FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...","English,French,Italian,German,Spanish - Spain,...",54.0,Action,"About This Game Developed by id software, the...",NaN,"Minimum:,OS:,Windows 7/8.1/10 (64-bit versions...","Recommended:,OS:,Windows 7/8.1/10 (64-bit vers...",$19.99,$14.99
1,https://store.steampowered.com/app/578080/PLAY...,app,PLAYERUNKNOWN'S BATTLEGROUNDS,PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roya...,"Mixed,(6,214),- 49% of the 6,214 user reviews ...","Mixed,(836,608),- 49% of the 836,608 user revi...","Dec 21, 2017",PUBG Corporation,"PUBG Corporation,PUBG Corporation","Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats","English,Korean,Simplified Chinese,French,Germa...",37.0,"Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,Mature Content Description The developers de...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,NaN
2,https://store.steampowered.com/app/637090/BATT...,app,BATTLETECH,Take command of your own mercenary outfit of '...,"Mixed,(166),- 54% of the 166 user reviews in t...","Mostly Positive,(7,030),- 71% of the 7,030 use...","Apr 24, 2018",Harebrained Schemes,"Paradox Interactive,Paradox Interactive","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Single-player,Multi-player,Online Multi-Player...","English,French,German,Russian",128.0,"Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$39.99,NaN
3,https://store.steampowered.com/app/221100/DayZ/,app,DayZ,The post-soviet country of Chernarus is struck...,"Mixed,(932),- 57% of the 932 user reviews in t...","Mixed,(167,115),- 61% of the 167,115 user revi...","Dec 13, 2018",Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...","English,French,Italian,German,Spanish - Spain,...",NaN,"Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,NaN,"Minimum:,OS:,Windows 7/8.1 64-bit,Processor:,I...","Recommended:,OS:,Windows 10 64-bit,Processor:,...",$44.99,NaN
4,https://store.steampowered.com/app/8500/EVE_On...,app,EVE Online,EVE Online is a community-driven spaceship MMO...,"Mixed,(287),- 54% of the 287 user reviews in t...","Mostly Positive,(11,481),- 74% of the 11,481 u...","May 6, 2003",CCP,"CCP,CCP","Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...","English,German,Russian,French",NaN,"Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,NaN,"Minimum:,OS:,Windows 7,Processor:,Intel Dual C...","Recommended:,OS:,Windows 10,Processor:,Intel i...",Free,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40833 entries, 0 to 40832
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   url                       40833 non-null  object 
 1   types                     40831 non-null  object 
 2   name                      40817 non-null  object 
 3   desc_snippet              27612 non-null  object 
 4   recent_reviews            2706 non-null   object 
 5   all_reviews               28470 non-null  object 
 6   release_date              37654 non-null  object 
 7   developer                 40490 non-null  object 
 8   publisher                 35733 non-null  object 
 9   popular_tags              37888 non-null  object 
 10  game_details              40313 non-null  object 
 11  languages                 40797 non-null  object 
 12  achievements              12194 non-null  float64
 13  genre                     40395 non-null  object 
 14  game_d

In [6]:
#variables we will use
df1= df[["name", "desc_snippet", "popular_tags","genre","original_price"]]

#delete missing values
df2 = pd.DataFrame(df1.dropna())

df2.head(10)


,name,desc_snippet,popular_tags,genre,original_price
0,DOOM,Now includes all three premium DLC packs (Unto...,"FPS,Gore,Action,Demons,Shooter,First-Person,Gr...",Action,$19.99
1,PLAYERUNKNOWN'S BATTLEGROUNDS,PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roya...,"Survival,Shooter,Multiplayer,Battle Royale,PvP...","Action,Adventure,Massively Multiplayer",$29.99
2,BATTLETECH,Take command of your own mercenary outfit of '...,"Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Action,Adventure,Strategy",$39.99
3,DayZ,The post-soviet country of Chernarus is struck...,"Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Action,Adventure,Massively Multiplayer",$44.99
4,EVE Online,EVE Online is a community-driven spaceship MMO...,"Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Action,Free to Play,Massively Multiplayer,RPG,...",Free
6,Devil May Cry 5,"The ultimate Devil Hunter is back in style, in...","Action,Hack and Slash,Great Soundtrack,Demons,...",Action,$59.99
7,Human: Fall Flat,Human: Fall Flat is a quirky open-ended physic...,"Funny,Multiplayer,Co-op,Puzzle,Physics,Local C...","Adventure,Indie",$14.99
8,They Are Billions,They Are Billions is a Steampunk strategy game...,"Early Access,Base Building,Strategy,Zombies,Su...","Strategy,Early Access",$29.99
9,Warhammer: Chaosbane,In a world ravaged by war and dominated by mag...,"RPG,Adventure,Hack and Slash,Action,Action RPG...","Action,Adventure,RPG",$49.99
10,For The King,For The King is a strategic RPG that blends ta...,"RPG,Turn-Based Combat,Adventure,Online Co-Op,C...","Adventure,Indie,RPG,Strategy",$19.99


In [7]:
#TF-IDF Matrix 

#Cleaning stop words 
tfidf = TfidfVectorizer(stop_words="english")

tfidf_matrix = tfidf.fit_transform(df2["desc_snippet"])

tfidf_matrix.shape



tfidf_matrix.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [9]:
tfidf.get_feature_names_out()

tfidf_matrix.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [11]:
#Cosine Similarity Matrix  

cosine_sim = cosine_similarity(tfidf_matrix,
                               tfidf_matrix)

cosine_sim.shape
cosine_sim[1]

array([0.        , 1.        , 0.        , ..., 0.        , 0.02683434,
       0.        ])

In [12]:
#Recommendation 

indices = pd.Series(df2.index, index=df2['name'])

indices.index.value_counts()

indices = indices[~indices.index.duplicated(keep='last')]



indices["DayZ"]

#indices["Sherlock Holmes"]

game_index = indices["DayZ"]

cosine_sim[game_index]

similarity_scores = pd.DataFrame(cosine_sim[game_index],columns=["score"])

game_indices = similarity_scores.sort_values("score", ascending=False)[1:11].index

df2["name"].iloc[game_indices]

14847                SteelLIFE
6698                 SurvivalZ
25529    Zombie City Defense 2
26440      Master of Mutations
34327               Torn Earth
21921              Virus Petya
28921                   NightZ
18786             Cube Defense
30308               The Region
2014                     EBOLA
Name: name, dtype: object

In [13]:
#Creating Game Recommendation Function 

def steam_game_recommender(title, cosine_sim, dataframe):
    indices = pd.Series(dataframe.index, index=dataframe['name'])
    indices = indices[~indices.index.duplicated(keep='last')]
    game_index = indices[title]
    similarity_scores = pd.DataFrame(cosine_sim[game_index], columns=["score"])
    game_indices = similarity_scores.sort_values("score", ascending=False)[1:11].index
    return dataframe['name'].iloc[game_indices]

In [14]:
steam_game_recommender("DayZ", cosine_sim, df2)

14847                SteelLIFE
6698                 SurvivalZ
25529    Zombie City Defense 2
26440      Master of Mutations
34327               Torn Earth
21921              Virus Petya
28921                   NightZ
18786             Cube Defense
30308               The Region
2014                     EBOLA
Name: name, dtype: object

In [15]:
steam_game_recommender("Street Fighter V", cosine_sim, df2)

2147           The Witcher Adventure Game
26947                            Lifeline
15304                                OESE
1871                      Monster Slayers
318      Thronebreaker: The Witcher Tales
15338                               Woven
21051                  Monster Loves You!
366                          DOOM Eternal
64                              Northgard
1737        Dark Future: Blood Red States
Name: name, dtype: object